In [1]:
import numpy as np
import matplotlib.pyplot as plt
import posfai2 as my
import time
import scipy.sparse as sp

In [2]:
# Define model
alphabet='ACGT'
alpha = len(alphabet)
L = 2; 
n_order=2;
n_adj=2
ohe_spec = my.get_ohe_spec_str(L, n_order=n_order, n_adj=n_adj)
seq = 'AC'
#seq = my.make_random_seqs(L=L, num_seqs=1, alphabet=alphabet)[0]

In [3]:
# Old function
t0 = time.perf_counter()
x_ohe_old = my.seq_to_x_ohe_old(seq=seq, ohe_spec_str=ohe_spec, sparse=True)
elapsed_time = time.perf_counter()-t0
print(f'Old function: {elapsed_time}')
x_old = np.array(x_ohe_old.todense().T, dtype=np.int64)
x_old

Old function: 0.0007932090000000169


array([[1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0]])

In [4]:
# New function
t0 = time.perf_counter()
x_ohe_new = my.seq_to_x_ohe(seq=seq, ohe_spec=ohe_spec, alphabet=alphabet)
elapsed_time = time.perf_counter()-t0
print(f'New function: {elapsed_time}')
x_new = x_ohe_new.todense().T
x_new

New function: 0.0001895840000000204


array([[1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0]])

In [6]:
# Make sure old and new x_ohe are the same
np.all(np.isclose(x_old,x_new))

True